In [0]:
storage_account_name = "cadfstorage"
storage_account_key = "Y8cR8X6SmC7QLojvNRw1h95qe94HGA1GbGDFJxRvs06wf9DVfuy/NaVCEQ0nOO/VyFH7x8KaccIr+AStY6uuYw=="

# Configure Spark to use the storage account key
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)



In [0]:
# Assuming both files are in CSV format
accounts_df = spark.read.format("csv").option("header", "true").load("abfss://bronze@cadfstorage.dfs.core.windows.net/accounts.csv")
customers_df = spark.read.format("csv").option("header", "true").load("abfss://bronze@cadfstorage.dfs.core.windows.net/customers.csv")
loan_payments_df = spark.read.format("csv").option("header", "true").load("abfss://bronze@cadfstorage.dfs.core.windows.net/loan_payments.csv")
loans_df = spark.read.format("csv").option("header", "true").load("abfss://bronze@cadfstorage.dfs.core.windows.net/loans.csv")
transactions_df = spark.read.format("csv").option("header", "true").load("abfss://bronze@cadfstorage.dfs.core.windows.net/transactions.csv")

accounts_df.show(5)
customers_df.show(5)
loan_payments_df.show(5)
loans_df.show(5)
transactions_df.show(5)

+----------+-----------+------------+-------+
|account_id|customer_id|account_type|balance|
+----------+-----------+------------+-------+
|         1|         45|     Savings|1000.50|
|         2|         12|    Checking|2500.75|
|         3|         78|     Savings|1500.00|
|         4|         34|    Checking|3000.25|
|         5|         56|     Savings| 500.00|
+----------+-----------+------------+-------+
only showing top 5 rows

+-----------+----------+---------+--------------+---------+-----+------+
|customer_id|first_name|last_name|       address|     city|state|   zip|
+-----------+----------+---------+--------------+---------+-----+------+
|          1|      John|      Doe|    123 Elm St|  Toronto|   ON|M4B1B3|
|          2|      Jane|    Smith| 456 Maple Ave|   Ottawa|   ON|K1A0B1|
|          3|   Michael|  Johnson|    789 Oak Dr| Montreal|   QC|H1A1A1|
|          4|     Emily|    Davis|   101 Pine Rd|  Calgary|   AB|T2A0A1|
|          5|     David|   Wilson|202 Birch Blvd|V

In [0]:
# Select only the customer_id column from the customer DataFrame
valid_customers = customers_df.select("customer_id").distinct()

# Join accounts with valid_customers on customer_id
cleaned_accounts_df = accounts_df.join(valid_customers, on="customer_id", how="inner")

cleaned_accounts_df.show(20)


+-----------+----------+------------+-------+
|customer_id|account_id|account_type|balance|
+-----------+----------+------------+-------+
|          7|        28|    Checking|2900.00|
|         51|        41|     Savings| 250.25|
|         15|        38|    Checking|3900.50|
|         54|        89|     Savings| 850.25|
|         11|        24|    Checking|2600.00|
|         29|        13|     Savings|1300.25|
|         69|        65|     Savings| 550.25|
|         42|        54|    Checking|5500.50|
|         73|        71|     Savings| 625.75|
|         87|        73|     Savings| 650.25|
|         64|        14|    Checking|3200.50|
|          3|        11|     Savings|1100.75|
|         30|        80|    Checking|8100.00|
|         34|         4|    Checking|3000.25|
|         59|        69|     Savings| 600.25|
|          8|        68|    Checking|6900.00|
|         22|        76|    Checking|7700.00|
|         28|        56|    Checking|5700.00|
|         85|        33|     Savin

In [0]:
cleaned_accounts_df.write.format("csv").mode("overwrite").option("header", "true").save("abfss://bronze@cadfstorage.dfs.core.windows.net/cleaned_accounts.csv")


In [0]:
cleaned_accounts_df.show(10)

+-----------+----------+------------+-------+
|customer_id|account_id|account_type|balance|
+-----------+----------+------------+-------+
|          7|        28|    Checking|2900.00|
|         51|        41|     Savings| 250.25|
|         15|        38|    Checking|3900.50|
|         54|        89|     Savings| 850.25|
|         11|        24|    Checking|2600.00|
|         29|        13|     Savings|1300.25|
|         69|        65|     Savings| 550.25|
|         42|        54|    Checking|5500.50|
|         73|        71|     Savings| 625.75|
|         87|        73|     Savings| 650.25|
+-----------+----------+------------+-------+
only showing top 10 rows



In [0]:
# Sort the DataFrame by 'accountid' in ascending order
cleaned_accounts_df_sorted = cleaned_accounts_df.orderBy("account_id")

# Write the sorted DataFrame to Azure Data Lake Storage in CSV format
cleaned_accounts_df_sorted.write.format("csv").mode("overwrite").option("header", "true").save("abfss://bronze@cadfstorage.dfs.core.windows.net/cleaned_accounts.csv")


In [0]:
cleaned_accounts_df_sorted.show(10)

+-----------+----------+------------+--------+
|customer_id|account_id|account_type| balance|
+-----------+----------+------------+--------+
|         45|         1|     Savings| 1000.50|
|         50|       100|    Checking|10100.00|
|          3|        11|     Savings| 1100.75|
|         81|        12|    Checking| 2700.00|
|         29|        13|     Savings| 1300.25|
|         64|        14|    Checking| 3200.50|
|         47|        15|     Savings|  700.75|
|         18|        16|    Checking| 1400.00|
|          5|        18|    Checking| 1600.50|
|         76|        19|     Savings|  400.75|
+-----------+----------+------------+--------+
only showing top 10 rows

